# Data Consistency between Server and Log

This notebook studies data consistency in the server.csv dataset and the relation between this and the connectivity_events.csv data.

In this notebook the file connectivity_events_clean01.csv will be used, resulting from the notebook [02_Data_Consistency](02_Data_Consistency.ipynb)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import aux_balena as aux
import importlib
from datetime import datetime, timedelta
%matplotlib inline

## Server

In [2]:
dfserver = aux.read_server()

Converting to timestamp field:  created_at
Converting to timestamp field:  destroyed_at
Done!


In [3]:
dfserver.head()

created_at            destroyed_at
server_id                                                   
2         2015-05-31 14:30:00.000000 2017-08-03 22:42:57.929
3         2017-08-03 22:43:07.392988 2017-08-03 22:46:50.279
4         2017-08-03 22:46:57.243687 2017-08-08 01:02:16.079
5         2017-08-08 01:02:26.761809 2017-08-29 19:57:59.039
6         2017-08-29 19:58:06.928351 2017-08-29 20:10:36.166

Check if creation and destruction times are compatible:

In [4]:
dfserver[dfserver.destroyed_at <= dfserver.created_at]

created_at            destroyed_at
server_id                                                   
22        2017-12-08 23:53:21.739516 2017-12-08 23:53:20.921
23        2017-12-08 23:55:50.180261 2017-12-08 23:55:49.203
24        2017-12-08 23:57:38.456028 2017-12-08 23:57:37.678

There are 3 servers where the destruction time occurs before the creation time.

These 3 servers are those missing in the connectivity_events database (see [01_EDA](01_EDA.ipynb))

The 3 servers will be removed in from this dataset

In [5]:
dfserver.drop(index=[22,23,24],inplace=True)

In [7]:
dfserver.to_csv('servers_clean01.csv',index=True)

## Server vs. Log consistency

In [8]:
dflog = aux.read_log('connectivity_events_clean01.csv')

Mem. usage decreased to 142.03 Mb (51.5% reduction)
Log read in 18.7 s
Converting to timestamp field:  timestamp
Done!


In [9]:
dflog.head()

timestamp  device_id  user_id  server_id  connected
0 2015-05-31 14:30:00          1        1          2       True
1 2015-06-02 10:30:00          1        1          2      False
2 2015-05-31 14:30:00          2        2          2       True
3 2015-06-05 20:30:00          2        2          2      False
4 2015-05-31 14:30:00          3        1          2       True

Merge dflog with dfserver to verify if datapoints exist after a server has been destroyed

In [10]:
dfcomp = pd.merge(dflog, dfserver,how='left',on='server_id',sort=False)
dfcomp.head()

timestamp  device_id  user_id  server_id  connected  \
0 2015-05-31 14:30:00          1        1          2       True   
1 2015-06-02 10:30:00          1        1          2      False   
2 2015-05-31 14:30:00          2        2          2       True   
3 2015-06-05 20:30:00          2        2          2      False   
4 2015-05-31 14:30:00          3        1          2       True   

           created_at            destroyed_at  
0 2015-05-31 14:30:00 2017-08-03 22:42:57.929  
1 2015-05-31 14:30:00 2017-08-03 22:42:57.929  
2 2015-05-31 14:30:00 2017-08-03 22:42:57.929  
3 2015-05-31 14:30:00 2017-08-03 22:42:57.929  
4 2015-05-31 14:30:00 2017-08-03 22:42:57.929

Condition of incompatibility:

In [11]:
condition = (dfcomp.connected==True) & (dfcomp.timestamp>dfcomp.destroyed_at)

In [12]:
sum(condition)

64

There 64 inconsistent datapoints between connectivity log and server data

In [14]:
# inconsistent points
df_inconsistent = dfcomp.copy()
df_inconsistent = df_inconsistent[condition]

In [18]:
# time difference between timestamp and server destruction time
df_inconsistent['delta']=df_inconsistent.timestamp - df_inconsistent.destroyed_at

In [19]:
df_inconsistent.head()

timestamp  device_id  user_id  server_id  connected  \
29991  2017-11-09 20:00:48.863         23       11         18       True   
110760 2018-01-08 13:18:55.414       3563       13         27       True   
351070 2017-12-08 23:35:18.598       7173      540         20       True   
388588 2018-01-08 13:18:56.075       7468      357         27       True   
393590 2018-01-08 13:18:53.485       7534      979         27       True   

                       created_at            destroyed_at           delta  
29991  2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998 00:00:01.865000  
110760 2017-12-13 17:03:53.912771 2018-01-08 13:18:51.058 00:00:04.356000  
351070 2017-11-22 16:15:57.285482 2017-12-08 23:35:12.164 00:00:06.434000  
388588 2017-12-13 17:03:53.912771 2018-01-08 13:18:51.058 00:00:05.017000  
393590 2017-12-13 17:03:53.912771 2018-01-08 13:18:51.058 00:00:02.427000

In [20]:
df_inconsistent.delta.describe()

count                        64
mean     0 days 00:00:03.744875
std      0 days 00:00:02.681607
min      0 days 00:00:00.043000
25%      0 days 00:00:01.784500
50%      0 days 00:00:03.523500
75%      0 days 00:00:05.437250
max      0 days 00:00:09.009000
Name: delta, dtype: object

The inconsistent connectivity events occur few seconds after server shutdown (max 9s)

In [21]:
len(np.unique(df_inconsistent.connected))

1

The inconsistent events are all of the same kind (ie. connected=True)

Checking the final part of the signal for a given inconsistent device...

In [25]:
k = 0
dev_id = df_inconsistent['device_id'].iloc[k] # device_id
s_id = df_inconsistent['server_id'].iloc[k] # server_id
dsub = dfcomp[(dfcomp.device_id==dev_id) & (dfcomp.server_id==s_id)]
dsub.tail()

timestamp  device_id  user_id  server_id  connected  \
29987 2017-11-09 19:41:14.608         23       11         18       True   
29988 2017-11-09 19:50:10.380         23       11         18      False   
29989 2017-11-09 19:50:57.432         23       11         18       True   
29990 2017-11-09 20:00:04.348         23       11         18      False   
29991 2017-11-09 20:00:48.863         23       11         18       True   

                      created_at            destroyed_at  
29987 2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998  
29988 2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998  
29989 2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998  
29990 2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998  
29991 2017-11-04 00:11:51.118728 2017-11-09 20:00:46.998

There seems to be a spuriuous ON signal generated immediately after the OFF signal occurred when the server is destroyed.

**REMEDY**: a new log file is created, in which these inconsistent datapoints have OFF status.

In [28]:
dfcomp_consistent = dfcomp.copy()
condition = (dfcomp_consistent.connected==True) &\
    (dfcomp_consistent.timestamp>dfcomp_consistent.destroyed_at)
dfcomp_consistent.loc[condition,['connected']] = False

In [29]:
# Validation
sum((dfcomp_consistent.connected==True) &\
    (dfcomp_consistent.timestamp>dfcomp_consistent.destroyed_at))

0

Create new csv file

In [30]:
dfcomp_consistent[['timestamp','device_id','user_id',\
                  'server_id','connected']].to_csv('connectivity_events_clean02.csv',\
                                                   index = False)